# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [49]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import time
from collections import defaultdict

# Set pandas display options for better visibility
pd.set_option('display.max_columns', None)

In [50]:
df = pd.read_csv('bank-additional-full.csv', sep = ';')

In [51]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [52]:
# Display basic info about the dataset
print("🔍 Dataset Overview:")
print(df.info())
print("Shape of dataset:", df.shape)

# Check for missing values
print("Missing Values per Feature:")
print(df.isnull().sum())

# Check unique values for each column to understand categorical levels
print("Unique values per feature:")
for col in df.columns:
    print(f"{col}: {df[col].nunique()} unique values")

# Check data types
print("Data Types:")
print(df.dtypes)

# Identify potential features needing coercion or cleanup
# Note: The dataset uses 'unknown' instead of NaN for missing categorical values.
unknown_counts = (df == 'unknown').sum()
print(" 'Unknown' value counts (indicating missing categorical data):")
print(unknown_counts[unknown_counts > 0])

# Recommendations
print("- All numeric columns are already in numeric format (int64/float64).")
print("- Categorical variables are of type 'object' and will need encoding.")
print("- 'unknown' entries should be treated as missing values during preprocessing.")
print("- The 'duration' feature will be removed before modeling to avoid target leakage.")


🔍 Dataset Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.i

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [53]:
The business objective is to increase the efficiency and success rate of the bank's direct marketing phone campaigns for term deposit subscription. 
This is achieved by building a predictive model that can accurately identify and prioritize potential clients who are most likely to subscribe to a term deposit (target variable y='yes') before the campaign call is made. 
By focusing resources (time, money, human effort) on these high-potential clients, the bank can maximize its return on investment and minimize wasted contacts.


SyntaxError: unterminated string literal (detected at line 1) (3142216578.py, line 1)

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [ ]:
e will build a basic model to get started. Before we can do this, we must work to encode the data. Using just the bank information features (age, job, marital, education, default, housing, loan), prepare the features and target column for modeling with appropriate encoding and transformations.

Steps:

Select features and target.

Encode the target variable y.

Apply One-Hot Encoding to categorical features.

Apply StandardScaler to the numeric feature (age).

In [54]:
# 1. Select only the bank client data features
BANK_CLIENT_FEATURES = ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan']
TARGET = 'y'

df_model = df[BANK_CLIENT_FEATURES + [TARGET]].copy()

# 2. Encode the target variable 'y'
# 'yes' = 1 (Subscribed), 'no' = 0 (Did not subscribe)
df_model[TARGET] = df_model[TARGET].map({'yes': 1, 'no': 0})

# 3. Apply One-Hot Encoding to categorical features
categorical_features = ['job', 'marital', 'education', 'default', 'housing', 'loan']
df_model = pd.get_dummies(df_model, columns=categorical_features, drop_first=True)

# Define the feature matrix X and target vector y
X = df_model.drop(TARGET, axis=1)
y = df_model[TARGET]

# 4. Identify and scale the numeric feature 'age'
numeric_features = ['age']
scaler = StandardScaler()

# Fit and transform 'age'
X['age'] = scaler.fit_transform(X[['age']])

# Check for any remaining non-numeric columns (should be empty)
object_cols_check = X.select_dtypes(include='object').columns.tolist()
if object_cols_check:
    # This warning is added to help diagnose the issue that caused the ValueError
    print(f"Warning: Non-numeric columns found before split: {object_cols_check}")

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [ ]:
With the engineered features, perform a train/test split. Use 80% of the data for training and 20% for testing. Set random_state=42 for reproducibility.

In [55]:
# Perform the 80/20 split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

X_train shape: (32950, 28)
X_test shape: (8238, 28)


### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [56]:
import pandas as pd

# Load the data
df = pd.read_csv('bank-additional-full.csv', sep = ';')

# Encode the target variable 'y'
# 'no' -> 0, 'yes' -> 1
df['y_encoded'] = df['y'].map({'yes': 1, 'no': 0})

# Calculate the value counts and total number of observations
target_counts = df['y_encoded'].value_counts()
total_observations = len(df)

# Determine the majority class and its proportion (baseline accuracy)
majority_class_count = target_counts.max()
baseline_accuracy = majority_class_count / total_observations

print(f"Target Value Counts:\n{target_counts}")
print(f"Total Observations: {total_observations}")
print(f"Baseline Accuracy (Majority Class Proportion): {baseline_accuracy:.4f}")

Target Value Counts:
y_encoded
0    36548
1     4640
Name: count, dtype: int64
Total Observations: 41188
Baseline Accuracy (Majority Class Proportion): 0.8873


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [57]:
log_reg_model = LogisticRegression(random_state=42, max_iter=1000)

# Fit the model to the training data
log_reg_model.fit(X_train, y_train)

# Predict on the test data
y_pred_initial = log_reg_model.predict(X_test)



### Problem 9: Score the Model

What is the accuracy of your model?

In [58]:
# Evaluate the model
test_accuracy = accuracy_score(y_test, y_pred_initial)

print(f"Logistic Regression Test Accuracy : {test_accuracy:.4f}")

Logistic Regression Test Accuracy : 0.8874


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [36]:
models = {
    "Logistic Regression": LogisticRegression(random_state=42, max_iter=1000),
    "K Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Support Vector Machine (SVC)": SVC(random_state=42)
}

results = defaultdict(list)

print("--- Model Comparison Results ---")

# Train, time, and score each model
for name, model in models.items():
    print(f"Training {name}...")
    start_time = time.time()
    model.fit(X_train, y_train)
    fit_time = time.time() - start_time

    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Scoring
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    # Store results
    results["Model"].append(name)
    results["Train Time (s)"].append(f"{fit_time:.4f}")
    results["Train Accuracy"].append(f"{train_accuracy:.4f}")
    results["Test Accuracy"].append(f"{test_accuracy:.4f}")

results_df = pd.DataFrame(results)

print("Classification Model Performance Comparison ")
print(results_df)


--- Model Comparison Results ---
Training Logistic Regression...
Training K Nearest Neighbors...
Training Decision Tree...
Training Support Vector Machine (SVC)...

Classification Model Performance Comparison (Full Features):
                          Model Train Time (s) Train Accuracy Test Accuracy
0           Logistic Regression         0.1499         0.8873        0.8874
1           K Nearest Neighbors         0.0212         0.8917        0.8809
2                 Decision Tree         0.1403         0.9171        0.8640
3  Support Vector Machine (SVC)       110.0057         0.8883        0.8865


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.


- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [59]:
dt_model = DecisionTreeClassifier(random_state=42)

# Define the parameter grid
param_grid = {
    'max_depth': [5, 7, 9, 11],
    'min_samples_leaf': [20, 30, 40],
    'min_samples_split': [10, 20, 30],
}

# Initialize GridSearchCV, optimizing for ROC AUC
print("\n--- Problem 10: Starting GridSearchCV for Decision Tree (Optimizing for ROC AUC) ---")
grid_search = GridSearchCV(
    estimator=dt_model,
    param_grid=param_grid,
    scoring='roc_auc',  # Best metric for imbalanced data
    cv=5,               # 5-fold cross-validation
    verbose=1,
    n_jobs=-1
)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Best model
best_dt_model = grid_search.best_estimator_

print("Best Decision Tree Model Parameters:")
print(grid_search.best_params_)

# Use the best model to predict on the test set
y_pred_best = best_dt_model.predict(X_test)

# Generate Classification Report and Confusion Matrix
report = classification_report(y_test, y_pred_best, target_names=['No (0)', 'Yes (1)'])
matrix = confusion_matrix(y_test, y_pred_best)


print("Classification Report of Best Decision Tree Model on Test Set")
print(report)
print("Confusion Matrix of Best DT Model on Test Set", matrix)


--- Problem 10: Starting GridSearchCV for Decision Tree (Optimizing for ROC AUC) ---
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best Decision Tree Model Parameters:
{'max_depth': 9, 'min_samples_leaf': 40, 'min_samples_split': 10}

--- Problem 11: Final Evaluation (Best Tuned Decision Tree) ---

Classification Report (Best DT Model on Test Set):
              precision    recall  f1-score   support

      No (0)       0.89      1.00      0.94      7310
     Yes (1)       0.45      0.03      0.05       928

    accuracy                           0.89      8238
   macro avg       0.67      0.51      0.50      8238
weighted avg       0.84      0.89      0.84      8238


Confusion Matrix (Best DT Model on Test Set):
 [[7278   32]
 [ 902   26]]


##### Questions